In [14]:
import pprint, hashlib, re

# salt = 'abc'
salt = 'zpqevtbw'
h = '0123456789abcdef'
quints = {k: {} for k in h}
triples = {k: {} for k in h}
keys = {}
hashes = {}

In [15]:
def gen_hash(y):
    m = hashlib.md5()
    m.update(y.encode())
    
    return m.hexdigest().lower()

def stretch_hash(y):
    for i in range(2017):
        m = hashlib.md5()
        m.update(y.encode())
        y = m.hexdigest().lower()

    return m.hexdigest().lower()

In [16]:
def check_keys(hashfn):
    index = 0
    while len(keys) < 64:
        for x in range(index, index + 5000):
            y = salt + str(x)
            hashes[x] = hashfn(y)

        for n in hashes:
            t = re.search(r'([a-f0-9])\1\1', hashes[n])
            if t:
                triples[t.group()[0]][n] = hashes[n]

            q = re.search(r'([a-f0-9])\1\1\1\1', hashes[n])
            if q:
                quints[q.group()[0]][n] = hashes[n]
        
        for k in quints:
            qvals = quints[k]
            tvals = triples[k]
            for q in qvals:
                for t in tvals:
                    if q in range(t+1, t+1001):
                        keys[t] = tvals[t]
        
        index += 5000
    
    return sorted(list(keys.keys()))[63]

In [17]:
check_keys(stretch_hash)

22423